In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
import math

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .getOrCreate()

"""
Reference gresearch:
- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/
- GitHub Spark extension: https://github.com/G-Research/spark-extension
- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet
"""

'\nReference gresearch:\n- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/\n- GitHub Spark extension: https://github.com/G-Research/spark-extension\n- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet\n'

In [3]:
sc = spark.sparkContext

In [4]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache dataframes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [5]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> "DataFrame":
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [6]:
sdf_large = sdf_generator(10000000, 8)
sdf_small = sdf_generator(10000000, 100000)
sdf_small.show()

+---+----------+--------------------+--------+-------+------+
| id|      date|           timestamp|idstring|idfirst|idlast|
+---+----------+--------------------+--------+-------+------+
|  0|2024-03-21|2024-03-21 05:40:...|       0|      0|     0|
|  1|2024-03-21|2024-03-21 05:40:...|       1|      1|     1|
|  2|2024-03-21|2024-03-21 05:40:...|       2|      2|     2|
|  3|2024-03-21|2024-03-21 05:40:...|       3|      3|     3|
|  4|2024-03-21|2024-03-21 05:40:...|       4|      4|     4|
|  5|2024-03-21|2024-03-21 05:40:...|       5|      5|     5|
|  6|2024-03-21|2024-03-21 05:40:...|       6|      6|     6|
|  7|2024-03-21|2024-03-21 05:40:...|       7|      7|     7|
|  8|2024-03-21|2024-03-21 05:40:...|       8|      8|     8|
|  9|2024-03-21|2024-03-21 05:40:...|       9|      9|     9|
| 10|2024-03-21|2024-03-21 05:40:...|      10|      1|     0|
| 11|2024-03-21|2024-03-21 05:40:...|      11|      1|     1|
| 12|2024-03-21|2024-03-21 05:40:...|      12|      1|     2|
| 13|202

In [7]:
sdf_small

DataFrame[id: bigint, date: date, timestamp: timestamp, idstring: string, idfirst: string, idlast: string]

In [8]:

#https://vincent.doba.fr/posts/20211004_spark_data_description_language_for_defining_spark_schema/
ddl_schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"

spark_schema = t.StructType(
    [
        t.StructField('id', t.LongType(), True), 
        t.StructField('date', t.DateType(), True), 
        t.StructField('timestamp', t.TimestampType(), True), 
        t.StructField('idstring', t.StringType(), True), 
        t.StructField('idfirst', t.StringType(), True), 
        t.StructField('idlast', t.StringType(), True)
    ]
)

# 1 - JSON

## 1-1 Save JSON

Results:
- Write time: small 11 min; big 3 s
- Data size: 1218 MB

How it works: https://github.com/jerryshao/apache-spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/DataFrameReader.scala#L323
https://spark.apache.org/docs/latest/sql-data-sources-json.html

In [9]:
sc.setJobDescription("Save Json Small")
path_json_small = "D:/Spark/Data/format_json_small.json"
#sdf_small.write.format("json").mode("overwrite").save(path_json_small)

In [10]:
sc.setJobDescription("Save Json Large")
path_json_large = "D:/Spark/Data/format_json_large.json"
#sdf_large.write.format("json").mode("overwrite").save(path_json_large)

In [11]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 160
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

## 1-2 Load without schema

Results:
- Small:
    - Listing leaf files and directories: 19 s
    - Schema interference: 5,8 min
    - Load data 1,3 min
    - Total: 7,1 min
- Big: 
    - Listing leaf files and directories: 0 s
    - Schema interference: 2 s
    - Load data 4 s
    - Total: 6 s

In [12]:
sc.setJobDescription("Load Json Small without schema")
sdf_json = spark.read.format("json").load(path_json_small)
sdf_json.write.format("noop").mode("overwrite").save()

In [13]:
sc.setJobDescription("Load Json Large without schema")
sdf_json = spark.read.format("json").load(path_json_large)
sdf_json.write.format("noop").mode("overwrite").save()

## 1-3 Load with Auto Schema

Results:
- Small:
    - Load data 5,4 min
- Big: 
    - Load data 4 s


In [14]:
schema = "id bigint, date string, timestamp string, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Json Small with auto schema")
sdf_json = spark.read.format("json").schema(schema).load(path_json_small)
sdf_json.write.format("noop").mode("overwrite").save()

In [15]:
schema = "id bigint, date string, timestamp string, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Json Large with auto schema")
sdf_json = spark.read.format("json").schema(schema).load(path_json_large)
sdf_json.write.format("noop").mode("overwrite").save()

## 1-4 Load with Target Schema

Results:
- Small:
    - Load data 5,7 min
- Big: 
    - Load data 7 s

In [16]:
schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Json Small with target schema")
sdf_json = spark.read.format("json").schema(schema).load(path_json_small)
sdf_json.write.format("noop").mode("overwrite").save()

In [17]:
schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Json Large with target schema")
sdf_json = spark.read.format("json").schema(schema).load(path_json_large)
sdf_json.write.format("noop").mode("overwrite").save()

# 2 - CSV with schema interference

## 2-1 Save CSV

Results:
- Write time: small 8,3 min; big 3 s
- Data size: small: 593 MB

How it works: https://github.com/jerryshao/apache-spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/DataFrameReader.scala#L470
https://spark.apache.org/docs/latest/sql-data-sources-csv.html

In [18]:
sc.setJobDescription("Save CSV Small")
path_csv_small = "D:/Spark/Data/format_csv_small.json"
#sdf_small.write.format("csv").mode("overwrite").save(path_csv_small)

In [19]:
sc.setJobDescription("Save CSV Large")
path_csv_large = "D:/Spark/Data/format_csv_large.json"
#sdf_large.write.format("csv").mode("overwrite").save(path_csv_large)

In [20]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 80
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

## 2-2 Load without schema

- Small:
    - Listing leaf files and directories: 25 s
    - Schema interference: 29 ms + 6,4 min
    - Load data 36 s
    - Total: 6,5 min
- Big: 
    - Listing leaf files and directories: 0 s
    - Schema interference: 26 ms + 6 s
    - Load data 7 s
    - Total: 14 s

In [21]:
sc.setJobDescription("Load CSV Small without schema")
sdf_csv = spark.read.format("csv").options(inferSchema=True, header=True).load(path_csv_small)
sdf_csv.write.format("noop").mode("overwrite").save()

In [22]:
sc.setJobDescription("Load CSV Large without schema")
sdf_csv = spark.read.format("csv").options(inferSchema=True, header=True).load(path_csv_large)
sdf_csv.write.format("noop").mode("overwrite").save()

## 2-3 Load with Auto Schema

- Small:
    - Load data 36 s
- Big: 
    - Load data 7 s


In [23]:
schema = "id int, date date, timestamp timestamp, idstring int, idfirst int, idlast int"
sc.setJobDescription("Load CSV Small with auto schema")
sdf_csv = spark.read.format("csv").options(header=True).schema(schema).load(path_csv_small)
sdf_csv.write.format("noop").mode("overwrite").save()

In [24]:
schema = "id int, date date, timestamp timestamp, idstring int, idfirst int, idlast int"
sc.setJobDescription("Load CSV Large with auto schema")
sdf_csv = spark.read.format("csv").options(header=True).schema(schema).load(path_csv_large)
sdf_csv.write.format("noop").mode("overwrite").save()

## 2-4 Load with Target schema

- Small:
    - Load data 33 s
- Big: 
    - Load data 7 s


In [25]:
schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load CSV Small with target schema")
sdf_csv = spark.read.format("csv").options(header=True).schema(schema).load(path_csv_small)
sdf_csv.write.format("noop").mode("overwrite").save()

In [26]:
schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load CSV Large with target schema")
sdf_csv = spark.read.format("csv").options(header=True).schema(schema).load(path_csv_large)
sdf_csv.write.format("noop").mode("overwrite").save()

# 3 - Parquet

## 3-1 Save Parquet

Results:
- Write time: small 9,3 min; big 2 s
- Data size: 81.5 MB

How it works: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [27]:
sc.setJobDescription("Save Parquet Small")
path_parquet_small = "D:/Spark/Data/format_parquet_small.parquet"
#sdf_small.write.format("parquet").mode("overwrite").save(path_parquet_small)

In [28]:
sc.setJobDescription("Save Parquet Lrge")
path_parquet_large = "D:/Spark/Data/format_parquet_large.parquet"
#sdf_large.write.format("parquet").mode("overwrite").save(path_parquet_large)

In [29]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 15
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

## 3-2 Load without schema

- Small:
    - Schema interference: 71 ms
    - Load data 5,6 min
    - Total: 5,6 min
- Big: 
    - Schema interference: 50 ms
    - Load data 1 s
    - Total: 1 s

In [30]:
sc.setJobDescription("Load Parquet Small without schema")
sdf_parquet = spark.read.format("parquet").load(path_parquet_small)
sdf_parquet.write.format("noop").mode("overwrite").save()

In [31]:
sc.setJobDescription("Load Parquet Large without schema")
sdf_parquet = spark.read.format("parquet").load(path_parquet_large)
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-3 Load with Schema

- Small:
    - Load data 4,7 min
- Big: 
    - Load data 0,9 s


In [32]:
sdf_schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Parquet Small with schema")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet_small)
sdf_parquet.write.format("noop").mode("overwrite").save()

In [33]:
sdf_schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Parquet Large with schema")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet_large)
sdf_parquet.write.format("noop").mode("overwrite").save()

# 4 - Avro

## 4-1 Save AVRO

Results:
- Write time: small 27 min; big 1 s
- Data size: 69.2 MB

How it works: https://spark.apache.org/docs/latest/sql-data-sources-avro.html

In [34]:
sc.setJobDescription("Save Avro Small")
path_avro_small = "D:/Spark/Data/format_avro_small.avro"
#sdf_small.write.format("avro").mode("overwrite").save(path_avro_small)

In [35]:
sc.setJobDescription("Save Avro Large")
path_avro_large = "D:/Spark/Data/format_avro_large.avro"
#sdf_large.write.format("avro").mode("overwrite").save(path_avro_large)

In [36]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 10
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

## 4-2 Load without schema

- Small:
    - Load data 7,1 min
- Big: 
    - Load data 8 s


In [37]:
sc.setJobDescription("Load Avro Small without schema")
sdf_avro = spark.read.format("avro").load(path_avro_small)
sdf_avro.write.format("noop").mode("overwrite").save()

In [38]:
sc.setJobDescription("Load Avro Large without schema")
sdf_avro = spark.read.format("avro").load(path_avro_large)
sdf_avro.write.format("noop").mode("overwrite").save()

## 4-3 Load with schema

- Small:
    - Load data 3,9 min
- Big: 
    - Load data 2 s

In [39]:
sdf_schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Avro Small with Schema")
sdf_avro = spark.read.format("avro").schema(sdf_schema).load(path_avro_small)
sdf_avro.write.format("noop").mode("overwrite").save()

In [40]:
sdf_schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"
sc.setJobDescription("Load Avro Large with Schema")
sdf_avro = spark.read.format("avro").schema(sdf_schema).load(path_avro_large)
sdf_avro.write.format("noop").mode("overwrite").save()